In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from optbinning import OptimalBinning
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegressionCV
from itertools import combinations
import gc


In [7]:
# Setting file paths

current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

# Loading data

FILE_PATH_DATA = project_root / 'data' / 'raw' / 'accepted_2007_to_2018Q4.csv.gz'
FILE_PATH_DICTIONARY = project_root / 'data' / 'raw'/ 'dictionary_approved_raw.csv' 

df = pd.read_csv(FILE_PATH_DATA, compression = 'gzip', low_memory = False)
df_dictionary = pd.read_csv(FILE_PATH_DICTIONARY, usecols = [0,1], low_memory = False)

# Comparing Data to Dictionary

data_columns = set(df.columns)
dictionary_columns = set(df_dictionary.iloc[:, 0].astype(str).str.strip())

common = data_columns.intersection(dictionary_columns)
in_data_only = data_columns - dictionary_columns
in_dict_only = dictionary_columns - data_columns

print(f"Total columns in Dataset: {len(data_columns)}")
print(f"Total variables in Dictionary: {len(dictionary_columns)}")
print(f"Matches found: {len(common)}\n")

print("-" * 30)
print(f"Columns in DATA but not in Dictionary ({len(in_data_only)}):")
print(list(in_data_only))

print("-" * 30)
print(f"Columns in DICTIONARY but not in Data ({len(in_dict_only)}):")
print(list(in_dict_only))

# Make sure the names of the features match
df_dictionary.iloc[:, 0] = df_dictionary.iloc[:, 0].replace('verified_status_joint', 'verification_status_joint')
df_dictionary = df_dictionary.dropna()

# Save cleaned dictionary
FILE_PATH_DICTIONARY_CLEANED = project_root / 'data' / 'cleaned'/ 'dictionary_approved_cleaned.csv'

df_dictionary.to_csv(FILE_PATH_DICTIONARY_CLEANED)

FILE_PATH = project_root / 'data' / 'accepted_2007_to_2018Q4.csv.gz'

cols_to_keep = [
    # Target
    'loan_status',

    # Structural / Lender Features (Known at Origination)
    'term', 'loan_amnt', 'initial_list_status', 'application_type', 'installment',

    # Applicant Features (The core data)
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
    'purpose', 'dti', 'delinq_2yrs', 
    'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
    'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'mths_since_last_delinq',
    'fico_range_low', 'fico_range_high',

    # Dates
    'issue_d', 'earliest_cr_line'
]
df = df[cols_to_keep]

Total columns in Dataset: 151
Total variables in Dictionary: 152
Matches found: 150

------------------------------
Columns in DATA but not in Dictionary (1):
['verification_status_joint']
------------------------------
Columns in DICTIONARY but not in Data (2):
['nan', 'verified_status_joint']


In [8]:
# Turning all variables into int or float usable types

target_statuses = [
    'Fully Paid', 
    'Charged Off', 
    'Default', 
    'Does not meet the credit policy. Status:Fully Paid',
    'Does not meet the credit policy. Status:Charged Off'
]

df = df[df['loan_status'].isin(target_statuses)].copy()

# target
bad_indicators = [
    'Charged Off', 
    'Default', 
    'Does not meet the credit policy. Status:Charged Off',
]
df['target'] = np.where(df['loan_status'].isin(bad_indicators), 1, 0)
df = df.drop(columns=['loan_status'])

term_map = {' 36 months': 1, ' 60 months': 2}
df['term_rank'] = df['term'].map(term_map)
df = df.drop(columns=['term'])

initial_list_status_map = {'w': 0, 'f': 1}
df['initial_list_status_f'] = df['initial_list_status'].map(initial_list_status_map)
df = df.drop(columns=['initial_list_status'])

application_type_map = {'Joint App': 0, 'Individual': 1}
df['individual_application'] = df['application_type'].map(application_type_map)
df = df.drop(columns=['application_type'])

df['emp_length_years'] = df['emp_length'].str.replace('< 1', '0').str.extract(r'(\d+)')
df['emp_length_years'] = pd.to_numeric(df['emp_length_years'], errors='coerce')
df = df.drop(columns=['emp_length'])

dummy_cols = ['home_ownership', 'purpose', 'verification_status']
df = pd.get_dummies(df, columns=dummy_cols, drop_first=True, dtype=int)

# Engineering date to get time since first credit lite
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')
df['credit_hist_months'] = ((df['issue_d'] - df['earliest_cr_line']) / pd.Timedelta(days=30))

# Normalise column names
df.columns = df.columns.str.replace(' ', '_').str.lower()


categorical_columns = [
    'target',
    'term_rank',
    'initial_list_status_f',
    'individual_application',
    'emp_length_years',
    'home_ownership_mortgage', 
    'home_ownership_none',
    'home_ownership_other',
    'home_ownership_own',
    'home_ownership_rent',
    'verification_status_source_verified',
    'verification_status_verified',
    'purpose_credit_card',
    'purpose_debt_consolidation',
    'purpose_educational',
    'purpose_home_improvement',
    'purpose_house',
    'purpose_major_purchase',
    'purpose_medical',
    'purpose_moving',
    'purpose_other',
    'purpose_renewable_energy',
    'purpose_small_business',
    'purpose_vacation',
    'purpose_wedding'
]

missing_categorical_cols = [col for col in categorical_columns if col not in df.columns]

if missing_categorical_cols:
    print(f"Warning: The following columns were not found in df: {missing_categorical_cols}")
else:
    print("All columns present")
    print(df.dtypes)

All columns present
loan_amnt                                     float64
installment                                   float64
annual_inc                                    float64
dti                                           float64
delinq_2yrs                                   float64
inq_last_6mths                                float64
open_acc                                      float64
pub_rec                                       float64
revol_bal                                     float64
revol_util                                    float64
total_acc                                     float64
mort_acc                                      float64
pub_rec_bankruptcies                          float64
mths_since_last_delinq                        float64
fico_range_low                                float64
fico_range_high                               float64
issue_d                                datetime64[ns]
earliest_cr_line                       datetime64[ns]
target  

In [ ]:
# Computing feasible interaction terms

# Downcast to float32
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype(np.float32)

# Setup
target = df['target'].values # Ensure target is available
non_predictive = ['target', 'issue_d', 'earliest_cr_line']
features = [col for col in df.columns if col not in non_predictive]

# Parameters
CORR_THRESHOLD = 0.05
new_features_list = []
batch_count = 0

print(f"Starting iteration over {len(features)} base features...")

# Iterate, Compute, Screen, Discard
for v1, v2 in combinations(features, 2):
    col_v1 = df[v1].values
    col_v2 = df[v2].values
    
    interactions = {}
    
    # Product
    feat_mult = col_v1 * col_v2
    interactions[f'{v1}_x_{v2}'] = feat_mult
    
    # Discarding those with low correlation to keep memory usage low
    for name, data in interactions.items():
        # Configuration
        dtype = "categorical" if col in categorical_columns else "numerical"
        
        # Enforce Monotonicity for numerical variables
        optb = OptimalBinning(
            name=col, 
            dtype=dtype, 
            solver="cp",
            monotonic_trend="auto_asc_desc",
            min_bin_size = 0.05,
            max_pvalue = 0.05
        )

        # Fit the Binner
        try:
            optb.fit(df_train[col], df_train[target_col])
        except Exception as e:
            print(f"Skipping {col} due to error: {e}")
            continue

        # Store Results
        binning_process[col] = optb
        
        # Get IV
        binning_table = optb.binning_table.build()
        iv = binning_table.loc['Totals', 'IV']
        iv_summary.append({'Variable': col, 'IV': iv})
        
        # Only keep if correlation exceeds threshold (
        if abs(corr) > CORR_THRESHOLD:
            # Store as Series to preserve index, append to list
            new_features_list.append(pd.Series(data, index=df.index, name=name))

    # Explicitly clear temporary variables and run garbage collection every 100 pairs
    del col_v1, col_v2, interactions
    batch_count += 1
    if batch_count % 100 == 0:
        gc.collect()
        print(f"Processed {batch_count} pairs. Retained {len(new_features_list)} features so far.")

# Construct Final DataFrame
if new_features_list:
    df_new = pd.concat(new_features_list, axis=1)
    df = pd.concat([df, df_new], axis=1)
    print(f"Final shape: {df.shape}")
else:
    print("No features met the correlation threshold.")

Starting iteration over 41 base features...
Processed 100 pairs. Retained 10 features so far.
Processed 200 pairs. Retained 10 features so far.
Processed 300 pairs. Retained 10 features so far.
Processed 400 pairs. Retained 10 features so far.
Processed 500 pairs. Retained 19 features so far.
Processed 600 pairs. Retained 29 features so far.


C:\Users\david_97mazrf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\david_97mazrf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Processed 700 pairs. Retained 31 features so far.
Processed 800 pairs. Retained 32 features so far.
Final shape: (1348099, 76)


In [ ]:
# Fitting lasso regression to check which variables are useful

non_predictive = ['target', 'issue_d', 'earliest_cr_line']
drop_cols = [c for c in non_predictive if c in df.columns]

X = df.drop(columns=drop_cols)
y = df['target']

# Cleaning
X = X.replace([np.inf, -np.inf], 0).fillna(0)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Lasso Estimation
lasso = LogisticRegressionCV(
    Cs=10, 
    cv=3, 
    penalty='l1', 
    solver='saga', 
    scoring='roc_auc',
    max_iter=1000,
    n_jobs=-1,
    random_state=0,
)

lasso.fit(X_scaled, y)

# Feature Selection
coefs = lasso.coef_[0] 
selected_mask = coefs != 0
selected_features = X.columns[selected_mask]

print(f"Optimal C value: {lasso.C_[0]}")
print(f"Original feature count: {X.shape[1]}")
print(f"Features selected by Lasso: {len(selected_features)}")

# Create Final Subset DataFrame
df_lasso = df[selected_features.tolist() + ['target']].copy()

KeyboardInterrupt: 